In [2]:
import numpy as np
import pandas as pd
from sklearn import *
import random
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from datetime import datetime
from matplotlib import pyplot as plt

In [3]:
df1= pd.read_csv("G:/Goldman Sachs/Bond Liquidity Prediction/ML_Bond_metadata.csv")
df2=pd.read_csv("G:/Goldman Sachs/Bond Liquidity Prediction/sell.csv")
df3=pd.read_csv("G:/Goldman Sachs/Bond Liquidity Prediction/buy.csv")

In [4]:
df1.rename(columns={'isin': 'id'}, inplace=True)
df1['amtIssued']= np.sqrt(np.log(df1['amtIssued']))
df1['amtOutstanding']= np.sqrt(np.log(df1['amtOutstanding']))
df1.issueDate.replace(np.NaN,-1,inplace=True)
df1.maturity.replace(np.NaN,-1,inplace= True)
df1.issueDate=pd.to_datetime(df1.issueDate)
df1.maturity=pd.to_datetime(df1.maturity)
df1['issue_year']=df1.issueDate.dt.year
df1['issue_month']=df1.issueDate.dt.month
df1['issue_day']=df1.issueDate.dt.dayofweek
df1['maturity_year']=df1.maturity.dt.year
df1['maturity_month']=df1.maturity.dt.month
df1['maturity_day']=df1.maturity.dt.dayofweek
df1.couponFrequency.replace(np.NaN, -1,inplace= 1)
df1.ratingAgency1EffectiveDate.replace(np.NaN,-1,inplace=True)
df1.ratingAgency2EffectiveDate.replace(np.NaN,-1,inplace= True)
df1.ratingAgency1EffectiveDate=pd.to_datetime(df1.ratingAgency1EffectiveDate)
df1.ratingAgency2EffectiveDate=pd.to_datetime(df1.ratingAgency2EffectiveDate)
df1['ratingAgency1_year']=df1.ratingAgency1EffectiveDate.dt.year
df1['ratingAgency1_month']=df1.ratingAgency1EffectiveDate.dt.month
df1['ratingAgency1_day']=df1.ratingAgency1EffectiveDate.dt.dayofweek
df1['ratingAgency2_year']=df1.ratingAgency2EffectiveDate.dt.year
df1['ratingAgency2_month']=df1.ratingAgency2EffectiveDate.dt.month
df1['ratingAgency2_day']=df1.ratingAgency2EffectiveDate.dt.dayofweek
df1['Day_differ']=df1.maturity-df1.issueDate
df1['Day_differ']=(df1.Day_differ/np.timedelta64(1,'D')).astype(int)

In [5]:
for column in ['market','collateralType','couponType','industryGroup','industrySector','industrySubgroup','maturityType','securityType','paymentRank','ratingAgency1Rating','ratingAgency1Watch','ratingAgency2Rating','ratingAgency2Watch']:
    dummies = pd.get_dummies(df1[column])
    df1[dummies.columns] = dummies

In [6]:
df1.drop(['issueDate','market','collateralType','couponType','industryGroup','industrySector','industrySubgroup','maturity','maturityType','securityType','paymentRank','ratingAgency1Rating','ratingAgency1Watch','ratingAgency2Rating','ratingAgency2Watch','ratingAgency1EffectiveDate','ratingAgency2EffectiveDate'], axis=1, inplace=True)

In [7]:
df1.rename(columns={'144aFlag': 'Flag'}, inplace=True)
df1.Flag.replace('flag0',0,inplace= 1)
df1.Flag.replace('flag1',1,inplace= 1)

In [8]:
df3.drop(['side','date','time'],axis=1, inplace=True)
df3['buyvolume']= np.sqrt(np.log(df3['buyvolume']))

In [9]:
df3.Day.replace({'Mon' : 1, 'Tue' : 2, 'Wed': 3, 'Thu': 4, 'Fri': 5},inplace=1)

In [10]:
train_buy= pd.merge(df3, df1, how='left', on='id')

In [11]:
ID_col=['id']
target_col=['buyvolume']
cat_col=['issuer']
features=list(set(list(train_buy.columns))-set(ID_col)-set(target_col)-set(cat_col))

In [12]:
test_10= pd.read_csv("G:/Goldman Sachs/Bond Liquidity Prediction/test_10jun2016.csv")
test_13=pd.read_csv("G:/Goldman Sachs/Bond Liquidity Prediction/test_13jun2016.csv")
test_14=pd.read_csv("G:/Goldman Sachs/Bond Liquidity Prediction/test_14jun2016.csv")

In [33]:
from sklearn.ensemble import BaggingRegressor
from sklearn import linear_model

In [ ]:
cv=[]
lst=[]
kf=cross_validation.KFold(len(train_buy),n_folds=3,random_state=0)
for idx1,idx2 in kf:
    x_train,x_cv=train_buy[features].iloc[idx1],train_buy[features].iloc[idx2]
    y_train,y_cv=train_buy.buyvolume.iloc[idx1],train_buy.buyvolume.iloc[idx2]
    random.seed(100)
    rf =GradientBoostingRegressor()
    rf.fit(x_train, y_train)
    cv.extend(rf.predict(x_cv))
    lst.append(rf.predict(test_10[features]))
    print(metrics.mean_squared_error(rf.predict(x_cv),y_cv))

In [24]:
pred1_10jun_buy=pd.DataFrame({'id': test_10.id, 'buyvolume_10':np.average(lst,axis=0)})
pred1_10jun_buy_cv=pd.DataFrame({'id': train_buy.id,'buyvolume_10': cv})

In [25]:
pred1_10jun_buy.buyvolume_10=np.exp(np.power(pred1_10jun_buy.buyvolume_10,2))
pred1_10jun_buy.to_csv("C:/Users/Rajat/Desktop/GB_10jun_buyvolume.csv",index= False)

In [26]:
pred1_10jun_buy_cv.buyvolume_10=np.exp(np.power(pred1_10jun_buy_cv.buyvolume_10,2))
pred1_10jun_buy_cv.to_csv("C:/Users/Rajat/Desktop/GB_10jun_buyvolume_cv.csv",index= False)